In [425]:
import numpy as np

In [426]:
def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

In [427]:
# GRADED FUNCTION: rnn_cell_forward

def rnn_cell_forward(xt, a_prev, parameters):
    """
    Implements a single forward step of the RNN-cell as described in Figure (2)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, a_prev, xt, parameters)
    """
    
    # Retrieve parameters from "parameters"
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    ### START CODE HERE ### (≈2 lines)
    # compute next activation state using the formula given above
    a_next = np.tanh(np.dot(Wax, xt) + np.dot(Waa, a_prev) + ba)

    # compute output of the current cell using the formula given above
    yt_pred = softmax(np.dot(Wya, a_next) + by)
    ### END CODE HERE ###
    
    # store values you need for backward propagation in cache
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

In [428]:
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)
print("a_next[4] = ", a_next[4])
print("a_next.shape = ", a_next.shape)
print("yt_pred[1] =", yt_pred[1])
print("yt_pred.shape = ", yt_pred.shape)

a_next[4] =  [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape =  (5, 10)
yt_pred[1] = [0.17867403 0.00059564 0.00192797 0.00231747 0.0396566  0.00828257
 0.00990921 0.21155864 0.43903066 0.00713304]
yt_pred.shape =  (2, 10)


In [429]:
# GRADED FUNCTION: rnn_forward

def rnn_forward(x, a0, parameters):
    """
    Implement the forward propagation of the recurrent neural network described in Figure (3).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    parameters -- python dictionary containing:
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)

    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y_pred -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of caches, x)
    """
    
    # Initialize "caches" which will contain the list of all caches
    caches = []
    
    # Retrieve dimensions from shapes of x and Wy
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    ### START CODE HERE ###
    
    # initialize "a" and "y" with zeros (≈2 lines)
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
    
    # Initialize a_next (≈1 line)
    a_next = a0
    
    # loop over all time-steps
    for t in range(T_x):
        # Update next hidden state, compute the prediction, get the cache (≈1 line)
        a_next, yt_pred, cache = rnn_cell_forward(x[:,:,t], a_next, parameters)
        # Save the value of the new "next" hidden state in a (≈1 line)
        a[:,:,t] = a_next
        # Save the value of the prediction in y (≈1 line)
        y_pred[:,:,t] = yt_pred
        # Append "cache" to "caches" (≈1 line)
        caches.append(cache)
        
        print("a = ", a)
        print("y_pred =", y_pred)
        
    ### END CODE HERE ###
    
    # store values needed for backward propagation in cache
    caches = (caches, x)
    
    return a, y_pred, caches

In [430]:
x = np.array([
 [ [0], [0.475429975]],
 [[0.475429975], [1]],
 [[1], [0.427518428]],
 [[0.427518428], [0.771498771]],
 ])

hidden_unit = 5
time_step = 2

print(x.shape)

(4, 2, 1)


In [431]:
from sklearn.preprocessing import MinMaxScaler

np.random.seed(1)
# x = np.random.randn(5,1,2) # rows | col | time_steps
# x = np.array([[[0]],
#  [[0.475429975]],
#  [[1]],
#  [[0.427518428]],
#  [[0.771498771]],
#  ])

x = np.array([
 [[0, 0.475429975]],
 [[0.475429975, 1]],
 [[1, 0.427518428]],
 [[0.427518428, 0.771498771]],
 ])

# x = np.array([
#  [ [0], [0.475429975]],
#  [[0.475429975], [1]],
#  [[1], [0.427518428]],
#  [[0.427518428], [0.771498771]],
#  ])

hidden_unit = 5
time_step = 2

print(x.shape)

a0 = np.random.randn(hidden_unit,1) # hidden_unit | col
Waa = np.random.randn(hidden_unit,hidden_unit) # hidden units | hidden units
Wax = np.random.randn(hidden_unit,4) # hidden units | rows
Wya = np.random.randn(2,hidden_unit) # n_predict | hidden units
ba = np.random.randn(hidden_unit,1) # hidden units | 1
by = np.random.randn(2,1) # n_predict | 1
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a, y_pred, caches = rnn_forward(x, a0, parameters)
# print("a.shape = ", a.shape)
# print("y_pred.shape = ", y_pred.shape)
# print("caches[1][1][0] =", caches[1][1][0])
# print("len(caches) = ", len(caches))

# print(a)

(4, 1, 2)
a =  [[[-0.99999817  0.        ]]

 [[ 0.99996147  0.        ]]

 [[-0.43331753  0.        ]]

 [[-0.9848844   0.        ]]

 [[ 0.96903934  0.        ]]]
y_pred = [[[0.16134821 0.        ]]

 [[0.83865179 0.        ]]]
a =  [[[-0.99999817  0.88413849]]

 [[ 0.99996147 -0.85682758]]

 [[-0.43331753  0.99601615]]

 [[-0.9848844   0.99266241]]

 [[ 0.96903934  0.99989989]]]
y_pred = [[[0.16134821 0.01435299]]

 [[0.83865179 0.98564701]]]


In [432]:
for arr in x:
    print(", ".join(str(x) for x in arr))

[0.         0.47542998]
[0.47542998 1.        ]
[1.         0.42751843]
[0.42751843 0.77149877]


In [433]:
for arr in a0:
    print(", ".join(str(x) for x in arr))

1.6243453636632417
-0.6117564136500754
-0.5281717522634557
-1.0729686221561705
0.8654076293246785


In [434]:
for arr in Waa:
    print(", ".join(str(x) for x in arr))

-2.3015386968802827, 1.74481176421648, -0.7612069008951028, 0.31903909605709857, -0.2493703754774101
1.462107937044974, -2.060140709497654, -0.3224172040135075, -0.38405435466841564, 1.1337694423354374
-1.0998912673140309, -0.17242820755043575, -0.8778584179213718, 0.04221374671559283, 0.5828152137158222
-1.1006191772129212, 1.1447237098396141, 0.9015907205927955, 0.5024943389018682, 0.9008559492644118
-0.6837278591743331, -0.12289022551864817, -0.9357694342590688, -0.2678880796260159, 0.530355466738186


In [435]:
for arr in Wax:
    print(", ".join(str(x) for x in arr))

-0.691660751725309, -0.39675352685597737, -0.6871727001195994, -0.8452056414987196
-0.671246130836819, -0.01266459891890136, -1.1173103486352778, 0.23441569781709215
1.6598021771098705, 0.7420441605773356, -0.19183555236161492, -0.8876289640848363
-0.7471582937508376, 1.6924546010277466, 0.05080775477602897, -0.6369956465693534
0.19091548466746602, 2.100255136478842, 0.12015895248162915, 0.6172031097074192


In [436]:
for arr in Wya:
    print(", ".join(str(x) for x in arr))

0.3001703199558275, -0.35224984649351865, -1.1425181980221402, -0.3493427224128775, -0.2088942333747781
0.5866231911821976, 0.8389834138745049, 0.9311020813035573, 0.2855873252542588, 0.8851411642707281


In [437]:
for arr in ba:
    print(", ".join(str(x) for x in arr))

-0.7543979409966528
1.2528681552332879
0.5129298204180088
-0.29809283510271567
0.48851814653749703


In [438]:
for arr in by:
    print(", ".join(str(x) for x in arr))

-0.07557171302105573
1.131629387451427
